# Create Map of a stream (based on NWIS Site ID) with various feature layers derived from USGS NLDI navigation along the stream, its tributaries and associated drainage basins

### TODO:  
* Put data (and/or URIs) in database and/or lists as they are retrieved
* Query NWIS Sites
 * Provide Input Widget for: 
  * NWIS Site search parameters
  * Distance
* How to locate HUC12s that share same PP?
* Locate NOAA weather stations within HUC (needs API key)

``
     url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/stations?datasetid=GHCND&locationid=HUC:050901&datatypeid=PRCP&startdate=2017-10-01&limit=50"
   headers = {"token":"dSWlkVrvxNPUSVBGXOeAWkynJQdJOxAB","Content-Type":"application/json;charset=UTF-8"}
   resp = requests.get(url, headers=headers)
   json_data = json.loads(resp.text)
   stations_050901 = pd.read_json(json.dumps(json_data['results']))
``

* Generate interactive plots after map
 * Try Seaborn or Bokeh (in addition to Pandas and Matplotlib)
 * Try Scipy.stats, Statsmodels or SKLearn for statistical modeling
* Set up deployment:
 * GitHub repo
 * Dockerfile
 * DockerHub
 * Autobuild Docker image from GitHub
 * MyBinder project with one-click build/test on GitHub


## Get things set up

#### To install any packages locally (e.g., folium, geopandas, fiona):
```
%%bash -l
use -e -r anaconda3-5.1
pip install folium --prefix=. --upgrade-strategy only-if-needed
```

#### To access them:
```
import os, sys
sys.path.insert(0,'lib/python3.7/site-packages') # Add locally installed packages to path
```

### Define the Reach by setting the starting site (NWIS Station) and up/down-stream distance ranges (Km)

In [176]:
reach = widgets.VBox(
    [
        widgets.Label(
            value='Select NWIS Station and range (Km):\n'
        ),
        widgets.Dropdown(
            options=[('Huntington','USGS-03206000'), ('Charleston','USGS-03198000')],
            value='USGS-03206000',
            description='NWIS Site:'
        ),
        widgets.IntSlider(
            value=5,
            min=0,
            max=100,
            step=5,
            description='Upstream:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d'
        ),
        widgets.IntSlider(
            value=5,
            min=0,
            max=100,
            step=5,
            description='Downstream:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d'
        )
    ]
)
display(reach)

In [182]:
NWIS_SITE = reach.children[1].value
UM_DIST = reach.children[2].value
DM_DIST = reach.children[3].value

In [188]:
print("NWIS Station: {0}, Upstream: {1}, Downstream: {2}".format(NWIS_SITE, UM_DIST, DM_DIST))

NWIS Station: USGS-03206000, Upstream: 5, Downstream: 5


### URLs for REST Web Services

In [112]:
USGS_NLDI_WS = "https://labs.waterdata.usgs.gov/api/nldi/linked-data" # USGS NLDI REST web services
NWIS_SITE_URL = USGS_NLDI_WS+"/nwissite/"+NWIS_SITE 
NWIS_SITE_NAV = NWIS_SITE_URL+"/navigate"
TNM_WS = "https://hydro.nationalmap.gov/arcgis/rest/services" # The National Map REST web services
ARCGIS_WS = "http://server.arcgisonline.com/arcgis/rest/services" # ARCGIS Online REST web services

### Input (optional) and Output Directories

In [113]:
IN_DIR = "data/"+NWIS_SITE
OUT_DIR = IN_DIR+"/out"

### Create output directory if it does not already exist

In [114]:
%mkdir -p {OUT_DIR}

In [115]:
import os.path
from os import path

### Use full width of browser window

In [116]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Enable numerical arrays and matrices with NumPy

In [117]:
import numpy as np

### Enable R-style DataFrames with Pandas

In [118]:
import pandas as pd

### Enable Math functions

In [119]:
import math

### Enable working with Dates and Times

In [189]:
import time
from datetime import datetime

### Enable inline MATLAB-style plotting with MatPlotLib

In [121]:
import matplotlib.pyplot as plt
%matplotlib inline

### Enable interactive plots using iPyWidgets

In [122]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

### Enable geospatial DataFrames with GeoPandas (built on Fiona, which is built on GDAL/OGR)

In [123]:
import geopandas as gpd

### Enable other geospatial functions using Shapely

In [124]:
import shapely
from shapely.geometry import Point, Polygon

### Enable Leatlet.JS-based mapping with Folium 

In [125]:
import folium
from folium import IFrame
import folium.plugins as plugins

### Enable additional HTML features with Branca

In [126]:
import branca

### Enable HTTP requests and parsing of JSON results

In [127]:
import requests
import json

### Enable SQL database access using SQLAlchemy (and GeoAlchemy)

In [128]:
from sqlalchemy import create_engine
from sqlalchemy import Table, MetaData

### Define some handy geometry functions

In [129]:
def geom_measure(lat1, lon1, lat2, lon2):
    R = 6378.137 # Radius of earth in KM
    dLat = lat2 * math.pi / 180 - lat1 * math.pi / 180
    dLon = lon2 * math.pi / 180 - lon1 * math.pi / 180
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(lat1 * math.pi / 180) * math.cos(lat2 * math.pi / 180) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    return d # Kilometers

In [130]:
def geom_extent(geom):
    lon1 = geom.total_bounds[0]
    lat1 = geom.total_bounds[1]
    lon2 = geom.total_bounds[2]
    lat2 = geom.total_bounds[3]
    d = geom_measure(lat1, lon1, lat2, lon2)
    return d

In [131]:
def geom_extent2(geom):
    d2 = geom_width(geom)+geom_height(geom)
    return d2

In [132]:
def geom_height(geom):
    lon1 = geom.total_bounds[0]
    lat1 = geom.total_bounds[1]
    lon2 = geom.total_bounds[2]
    lat2 = geom.total_bounds[3]
    h = geom_measure(lat1, lon1, lat2, lon1)
    return h

In [133]:
def geom_width(geom):
    lon1 = geom.total_bounds[0]
    lat1 = geom.total_bounds[1]
    lon2 = geom.total_bounds[2]
    lat2 = geom.total_bounds[3]
    w = geom_measure(lat1, lon1, lat1, lon2)
    return w

In [134]:
def geom_bbox(geom):
    polygon = gpd.GeoDataFrame(gpd.GeoSeries(geom.envelope), columns=['geometry'])
    return polygon

In [135]:
# In case CRS is different

def geom_bbox2(geom):
    lon_point_list = [geom.total_bounds[0],geom.total_bounds[2],geom.total_bounds[2],geom.total_bounds[0],geom.total_bounds[0]]
    lat_point_list = [geom.total_bounds[1],geom.total_bounds[1],geom.total_bounds[3],geom.total_bounds[3],geom.total_bounds[1]]
    polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
    crs = {'init': 'epsg:4326'}
    polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])    
    return polygon

In [136]:
# Try using the area of the total_bounds polygon in both degrees and meters to generate an approximate "conversion" factor

def geom_area(geom):
    factor = geom_width(geom)*geom_height(geom)/geom_bbox(geom).area
    area = factor*geom.area
    return area

In [137]:
# Use a rectangular projection to get true area
# *** Currently crashes kernel ***

def geom_area2(geom):
    geom_m = geom.to_crs(epsg=3857) # or 32633
    a = geom_m.area/10**6
    return a

### Set up a local file-based SQLite/Spatialite database
 * TODO:
  * Switch to Spatialite to hold geometry fields (instead of just Lat/Lon values)
  * Provide option to use PostgreSQL/PostGIS running in a separate Docker container (along with Adminer)
  * Make use of ORM model

In [138]:
sqlite_db_filename = 'riverwalk.db'
sqlite_engine = create_engine('sqlite:///{0}/{1}'.format(OUT_DIR,sqlite_db_filename))

### Create an optional Sites database table from an input CSV file
 * TODO: eliminate extraneous columns.  Currently have:
 'Type', 'Name', 'Longitude', 'Latitude', 'Elevation', 'River Code', 'Pool', 'RMI', 'Bank', 'State', 'County', 'HUC', 'Drainage Area', 'WQX Code', 'USGS Code', 'NWS Code', 'USACE Code', 'ORSANCO Code', 'Description'

In [139]:
optional_sites = path.exists(IN_DIR+'/sites.csv')

if (optional_sites):
    sites = pd.read_csv(IN_DIR+'/sites.csv',dtype={'County':object,'HUC':object,'WQX Code':object,'USGS Code':object})
    sites.to_sql("sites", con=sqlite_engine, index=False, if_exists='replace')

    meta = MetaData(bind=sqlite_engine)
    sqlite_engine._metadata = meta
    meta.reflect(sqlite_engine)

    sites_table = Table('sites',meta,autoload=True,autoload_with=sqlite_engine)

### Get Lat/Lon coordinates of starting site (NWIS station)

In [140]:
nwis_site_json = gpd.read_file(NWIS_SITE_URL)
nwis_site_geom = nwis_site_json.iloc[0]['geometry']
nwis_site_coord = [nwis_site_geom.y, nwis_site_geom.x]

## Generate map

In [141]:
river_map = folium.Map(nwis_site_coord,zoom_start=10,tiles=None)
plugins.ScrollZoomToggler().add_to(river_map)
plugins.Fullscreen(
    position='bottomright',
    title='Full Screen',
    title_cancel='Exit Full Screen',
    force_separate_button=True
).add_to(river_map)

### Add features (NWIS and WQP sites) from NLDI web services at USGS

In [142]:
# Popup parameters

width = 500
height = 120
max_width = 1000

# Main Stream

folium.GeoJson(NWIS_SITE_NAV+"/UM?distance="+str(UM_DIST),name="Main Stream (up)",show=True,control=False).add_to(river_map)
folium.GeoJson(NWIS_SITE_NAV+"/DM?distance="+str(DM_DIST),name="Main Stream (down)",show=True,control=False).add_to(river_map)

# NWIS Sites

fg_nwis = folium.FeatureGroup(name="USGS (NWIS) Sites",overlay=True,show=False)
color = 'darkred'
icon = 'dashboard'
        
nwis_sites_dm = gpd.read_file(NWIS_SITE_NAV+"/DM/nwissite?distance="+str(DM_DIST))
nwis_sites_um = gpd.read_file(NWIS_SITE_NAV+"/UM/nwissite?distance="+str(UM_DIST))
nwis_sites = gpd.GeoDataFrame(pd.concat([nwis_sites_dm,nwis_sites_um], ignore_index=True), crs=nwis_sites_dm.crs)

for i, nwis_site in nwis_sites.iterrows():
    coord = [nwis_site.geometry.y,nwis_site.geometry.x]
    label = 'NWIS Station: '+nwis_site.identifier+" ("+str(i)+")"
    html = label
    html += '<br>{0:s}'.format(nwis_site['name'])
    html += '<br><a href=\"{0:s}\",target=\"_blank\">{1:s}'.format(nwis_site.uri,nwis_site.uri)+'</a>'
    html += '<br>Lat: {0:.2f}'.format(nwis_site.geometry.y)+', Lon: {0:.2f}'.format(nwis_site.geometry.x)
    html += '<br>Comid: {0:s}'.format(nwis_site.comid)
    iframe = folium.IFrame(html,width=width,height=height)
    popup = folium.Popup(iframe,max_width=max_width)
    fg_nwis.add_child(folium.Marker(location=coord,icon=folium.Icon(color=color,icon=icon),popup=popup,tooltip=label))
        
fg_nwis.add_to(river_map)

# WQP Stations
        
fg_wqp = folium.FeatureGroup(name="WQP Stations",overlay=True,show=False)
color = 'darkgreen'
radius = 3
        
wqp_sites_dm = gpd.read_file(NWIS_SITE_NAV+"/DM/wqp?distance="+str(DM_DIST))
wqp_sites_um = gpd.read_file(NWIS_SITE_NAV+"/UM/wqp?distance="+str(UM_DIST))
wqp_sites = gpd.GeoDataFrame(pd.concat([wqp_sites_dm,wqp_sites_um], ignore_index=True), crs=wqp_sites_dm.crs)

for i, wqp_site in wqp_sites.iterrows():
    coord = [wqp_site.geometry.y,wqp_site.geometry.x]
    label = 'WQP Station: '+wqp_site.identifier+" ("+str(i)+")"
    html = label
    html += '<br>{0:s}'.format(wqp_site['name'])
    html += '<br><a href=\"{0:s}\",target=\"_blank\">{1:s}'.format(wqp_site.uri,wqp_site.uri)+'</a>'
    html += '<br>Lat: {0:.2f}'.format(wqp_site.geometry.y)+', Lon: {0:.2f}'.format(wqp_site.geometry.x)
    html += '<br>Comid: {0:s}'.format(wqp_site.comid)
    iframe = folium.IFrame(html,width=width,height=height)
    popup = folium.Popup(iframe,max_width=max_width)
    fg_wqp.add_child(folium.CircleMarker(location=coord,radius=radius,color=color,popup=popup,tooltip=label))

fg_wqp.add_to(river_map)

### Add HUC12 Pour Points (upstream and downstream) and *differential* drainage basins

In [143]:
fg_huc12pp = folium.FeatureGroup(name="HUC12 Pour Points",overlay=True,show=False)
fg_basins = folium.FeatureGroup(name="Drainage Basins",overlay=True,show=False)

color = 'darkblue'
radius = 3
        
huc12pp_site_dm = gpd.read_file(NWIS_SITE_NAV+"/DM/huc12pp?distance="+str(DM_DIST))
huc12pp_sites_um = gpd.read_file(NWIS_SITE_NAV+"/UM/huc12pp?distance="+str(UM_DIST))
huc12pp_sites = gpd.GeoDataFrame(pd.concat([huc12pp_site_dm,huc12pp_sites_um], ignore_index=True), crs=huc12pp_site_dm.crs)

n_segs = len(huc12pp_sites)-1

# Sort sites by decreasing area of drainage basin
def get_area(x):
    x_basin = gpd.read_file(USGS_NLDI_WS+"/comid/"+x+"/basin")
    return int(round(x_basin.iloc[0].geometry.area,3)*1000)  

huc12pp_sites['area']=huc12pp_sites.apply(lambda x: get_area(x.comid), axis=1)
huc12pp_sites.set_index(['area'],inplace=True,drop=True)
huc12pp_sites.sort_index(inplace=True,ascending=False)

i = 0
i_max = n_segs

for area, huc12pp_site in huc12pp_sites.iterrows():
    basin_url = USGS_NLDI_WS+"/comid/{0:s}/basin".format(huc12pp_site.comid)
    basin = gpd.read_file(basin_url)
    #basin_area = basin.iloc[0].geometry.area
    basin_area = geom_area(basin)
    basin_diff_area = basin_area
    wbd12_url = TNM_WS+"/wbd/MapServer/6/query?where=HUC12%3D%27{0:s}%27&outFields=NAME%2CHUC12%2CSHAPE_Length&f=geojson".format(huc12pp_site.identifier)
    wbd12 = gpd.read_file(wbd12_url)
    
    if i > 0:
        # Generate and show difference (polygon) between sussessive drainage basins
        basin_diff = gpd.overlay(basin_prev,basin,how='difference')  
        basin_diff_area = geom_area(basin_diff)
        style_function = lambda x: {'color': 'red', 'weight': 1, 'fillColor': 'blue', 'fillOpacity': 0.1}
        highlight_function = lambda x: {'color':'yellow', 'weight':3}
        tooltip = "Differential Drainage Basin for HUC12 Pour Point: {0:s} ({1:s}), Area: {2:.2f}".format(wbd12.iloc[0].HUC12,wbd12.iloc[0].NAME,basin_diff_area[0])
        basin_diff_feature = folium.GeoJson(basin_diff.iloc[0].geometry.buffer(-0.001).buffer(0.001),style_function=style_function,highlight_function=highlight_function,tooltip=tooltip)
        fg_basins.add_child(basin_diff_feature)
        
        if i == i_max:
            # Show large basin of first (highest upstream) pour point
            style_function = lambda x: {'color': 'gray', 'weight': 1, 'fillColor': 'gray', 'fillOpacity': 0.1}
            highlight_function = lambda x: {'color':'yellow', 'weight':1}
            tooltip = "Total Drainage Basin for HUC12 Pour Point: {0:s} ({1:s}), Area: {2:.2f}".format(wbd12.iloc[0].HUC12,wbd12.iloc[0].NAME,basin_area[0])
            basin_feature = folium.GeoJson(basin.iloc[0].geometry,style_function=style_function,highlight_function=highlight_function,tooltip=tooltip)
            fg_basins.add_child(basin_feature)
            
    basin_prev = basin

    # HUC12 Pour Point markers
    coord = [huc12pp_site.geometry.y,huc12pp_site.geometry.x]
    label = 'Pour Point for HUC12: '+wbd12.iloc[0].NAME
    html = label
    html += '<br>Indentifier: {0:s}'.format(huc12pp_site.identifier)
    html += '<br>Lat: {0:.2f}, Lon: {1:.2f}'.format(huc12pp_site.geometry.y,huc12pp_site.geometry.x)
    html += '<br>Comid: {0:s}'.format(huc12pp_site.comid)
    html += '<br>Area Total: {0:.2f}'.format(basin_area[0])
    html += '<br>Area Difference: {0:.2f}'.format(basin_diff_area[0])
    iframe = folium.IFrame(html,width=width,height=height)
    popup = folium.Popup(iframe,max_width=max_width)
    fg_huc12pp.add_child(folium.CircleMarker(location=coord,radius=radius,color=color,popup=popup,tooltip=label))
    
    i = i + 1

fg_huc12pp.add_to(river_map)
fg_basins.add_to(river_map)

### Add HUC12 and HUC10 WBD boundaries from The National Map web services (ArcGIS) for Pour Points along main stream and up nearby tributaries
 * TODO: Get HUC12s from up-tribs themselves instead the Pour Points

In [144]:
fg_wbd12 = folium.FeatureGroup(name="HUC12 Boundaries",overlay=True,show=False)
fg_wbd12_ut = folium.FeatureGroup(name="Up-Trib HUC12s",overlay=True,show=False)
fg_wbd10 = folium.FeatureGroup(name="HUC10 Boundaries",overlay=True,show=False)
        
i = 0
i_max = n_segs
huc12pp_comid_list = []
huc12_list = []
huc10_list = []

for area, huc12pp_site in huc12pp_sites.iterrows():
    if (huc12pp_site.identifier not in huc12_list):
        huc12_list.append(huc12pp_site.identifier)
    if (huc12pp_site.comid not in huc12pp_comid_list):
        huc12pp_comid_list.append(huc12pp_site.comid)

    # Get HUC12 watershed boundary (WBD) for pour point from The National Map web services
    wbd12_url = TNM_WS+"/wbd/MapServer/6/query?where=HUC12%3D%27{0:s}%27&outFields=NAME%2CHUC12%2CSHAPE_Length&f=geojson".format(huc12pp_site.identifier)
    wbd12 = gpd.read_file(wbd12_url)
    
    if i < i_max:
        style_function = lambda x: {'color': 'darkgreen', 'weight': 1, 'fillColor': 'green', 'fillOpacity': 0.1}
        highlight_function = lambda x: {'color':'yellow', 'weight':2}
        tooltip = "HUC12: {0:s} ({1:s}), Area: {2:.2f}".format(wbd12.iloc[0].HUC12,wbd12.iloc[0].NAME,geom_area(wbd12)[0])
        wbd12_feature = folium.GeoJson(wbd12.iloc[0].geometry,style_function=style_function,highlight_function=highlight_function,tooltip=tooltip)
        fg_wbd12.add_child(wbd12_feature)  

        # Get extent (width + height) of WBD associated with HUC12 Pour Point
        distance = int(round(geom_extent2(wbd12),0))

        # Find other pour points (and associated HUC12s) up stream (following tributares) within the current HUC12
        huc12pp_ut_url = USGS_NLDI_WS+"/comid/{0:s}/navigate/UT/huc12pp?distance={1:d}".format(huc12pp_site.comid,distance)
        huc12pp_ut_sites = gpd.read_file(huc12pp_ut_url)
        
        for k, huc12pp_ut in huc12pp_ut_sites.iterrows():
            if (huc12pp_ut.comid in huc12pp_comid_list):
                continue            
            else:
                huc12pp_comid_list.append(huc12pp_ut.comid)
            
            if (huc12pp_ut.identifier in huc12_list):
                continue                
            else:
                huc12_list.append(huc12pp_ut.identifier)
            
            # Get and display WBD for HUC12 associated with up-trib Pour Point
            wbd12_ut_url = TNM_WS+"/wbd/MapServer/6/query?where=HUC12%3D%27{0:s}%27&outFields=NAME%2CHUC12%2CSHAPE_Length&f=geojson".format(huc12pp_ut.identifier)
            wbd12_ut = gpd.read_file(wbd12_ut_url)
            style_function = lambda x: {'color': 'darkgreen', 'weight': 1, 'fillColor': 'green', 'fillOpacity': 0.05}
            highlight_function = lambda x: {'color':'yellow', 'weight':2}
            tooltip = "HUC12: {0:s} ({1:s}), Area: {2:.2f}".format(wbd12_ut.iloc[0].HUC12,wbd12_ut.iloc[0].NAME,geom_area(wbd12_ut)[0])
            wbd12_ut_feature = folium.GeoJson(wbd12_ut_url,style_function=style_function,highlight_function=highlight_function,tooltip=tooltip)
            fg_wbd12_ut.add_child(wbd12_ut_feature)        

            # Put marker at up-trib HUC12 Pour Point
            coord = [huc12pp_ut.geometry.y,huc12pp_ut.geometry.x]
            label = 'UT Pour Point for HUC12: '+wbd12_ut.iloc[0].NAME
            fg_wbd12_ut.add_child(folium.CircleMarker(location=coord,radius=4,color='lightblue',tooltip=label))

            # Get and display HUC10 containing HUC12
            huc10_identifier = huc12pp_ut.identifier[:-2]
            
            if (huc10_identifier in huc10_list):
                continue
            else:
                huc10_list.append(huc10_identifier)
                
            wbd10_url = TNM_WS+"/wbd/MapServer/5/query?where=HUC10%3D%27{0:s}%27&outFields=NAME%2CHUC10%2CSHAPE_Length&f=geojson".format(huc10_identifier)
            wbd10 = gpd.read_file(wbd10_url)
            style_function = lambda x: {'color': 'darkgreen', 'weight': 1, 'fillColor': 'green', 'fillOpacity': 0.05}
            highlight_function = lambda x: {'color':'yellow', 'weight':2}
            tooltip = "HUC10: {0:s} ({1:s}), Area: {2:.2f}".format(wbd10.iloc[0].HUC10,wbd10.iloc[0].NAME,geom_area(wbd10)[0])
            wbd10_feature = folium.GeoJson(wbd10_url,style_function=style_function,highlight_function=highlight_function,tooltip=tooltip)
            fg_wbd10.add_child(wbd10_feature)  
            
    i = i + 1

fg_wbd12.add_to(river_map)
fg_wbd12_ut.add_to(river_map)
fg_wbd10.add_to(river_map)

### Add tributaries upstream of HUC12 Pour Points

In [145]:
fg_utpp = folium.FeatureGroup(name="Tribs upstream of PPs",overlay=True,show=False)

for huc12 in huc12_list:
    wbd12_url = TNM_WS+"/wbd/MapServer/6/query?where=HUC12%3D%27{0:s}%27&f=geojson".format(huc12)
    wbd12 = gpd.read_file(wbd12_url)
    distance = int(round(geom_extent(wbd12),0))
    #distance = 35
    tribs = folium.GeoJson(USGS_NLDI_WS+"/huc12pp/{0:s}/navigate/UT?distance={1:d}".format(huc12,distance))
    fg_utpp.add_child(tribs)
    
fg_utpp.add_to(river_map)

### Add Bordering Counties
 * TODO: Get County codes from drainage basins

fg_counties = folium.FeatureGroup(name="Bordering Counties",overlay=True,show=False)
tiger_lines_url = "https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/State_County/MapServer/55/query?"
query_params = "&outFields=STATE%2C+COUNTY%2C+NAME&returnGeometry=true&f=geojson"

mason = folium.GeoJson(tiger_lines_url+"where=GEOID%3D%2754053%27"+query_params,tooltip='Mason') 
fg_counties.add_child(mason)

cabel = folium.GeoJson(tiger_lines_url+"where=GEOID%3D%2754011%27"+query_params,tooltip='Cabel') 
fg_counties.add_child(cabel)

wayne = folium.GeoJson(tiger_lines_url+"where=GEOID%3D%2754099%27"+query_params,tooltip='Wayne') 
fg_counties.add_child(wayne)

boyd = folium.GeoJson(tiger_lines_url+"where=GEOID%3D%2721019%27"+query_params,tooltip='Boyd') 
fg_counties.add_child(boyd)

greenup = folium.GeoJson(tiger_lines_url+"where=GEOID%3D%2721089%27"+query_params,tooltip='Greenup') 
fg_counties.add_child(greenup)

galia = folium.GeoJson(tiger_lines_url+"where=GEOID%3D%2739053%27"+query_params,tooltip='Galia') 
fg_counties.add_child(galia)

lawrence = folium.GeoJson(tiger_lines_url+"where=GEOID%3D%2739087%27"+query_params,tooltip='Lawrence') 
fg_counties.add_child(lawrence)

scioto = folium.GeoJson(tiger_lines_url+"where=GEOID%3D%2739145%27"+query_params,tooltip='Scioto') 
fg_counties.add_child(scioto)

fg_counties.add_to(river_map)

### Add NPDES Point Sources along tributaries
 * TODO: Check to see why NPDES data is no longer accessible

In [146]:
fg_npdes = folium.FeatureGroup(name="NPDES Point Sources",overlay=True,show=False)
color = 'orange'
radius = 2

i = 0
i_max = n_segs

for area, huc12pp_site in huc12pp_sites.iterrows():
    wbd12_url = TNM_WS+"/wbd/MapServer/6/query?where=HUC12%3D%27{0:s}%27&outFields=NAME%2CHUC12%2CSHAPE_Length&f=geojson".format(huc12pp_site.identifier)
    wbd12 = gpd.read_file(wbd12_url)
    distance = int(round(geom_extent(wbd12),0))
    
    try:
        npdes_sites = gpd.read_file(huc12pp_site.navigation+"/UT/npdes_rad?distance={0:d}".format(distance))
    except:
        i = i + 1
        continue
        
    if i < i_max:
        for j, npdes_site in npdes_sites.iterrows():
            coord = [npdes_site.geometry.y,npdes_site.geometry.x]
            label = 'NPDES Site: '+npdes_site.identifier
            html = label
            html += '<br>{0:s}'.format(npdes_site['name'])
            html += '<br><a href=\"{0:s}\",target=\"_blank\">{1:s}'.format(npdes_site.uri,npdes_site.uri)+'</a>'
            html += '<br>Lat: {0:.2f}'.format(npdes_site.geometry.y)+', Lon: {0:.2f}'.format(npdes_site.geometry.x)
            html += '<br>Comid: {0:s}'.format(npdes_site.comid)
            html += '<br>Reachcode: {0:s}'.format(npdes_site.reachcode)
            iframe = folium.IFrame(html,width=width,height=height)
            popup = folium.Popup(iframe,max_width=max_width)
            fg_npdes.add_child(folium.CircleMarker(location=coord,radius=radius,color=color,popup=popup,tooltip=label))
            
    i = i + 1
    
fg_npdes.add_to(river_map)

### Add optional input sites
 * TODO: Make more generic

In [147]:
if (optional_sites):
    sites_df = pd.read_sql_query('select * from sites', sqlite_engine)
    site_geometries = gpd.GeoSeries(sites_df.apply(lambda z: Point(z['Longitude'],z['Latitude']),1),crs={'init':'epsg:4326'})
    sites = gpd.GeoDataFrame(sites_df.drop(['Latitude','Longitude'],1),geometry=site_geometries)
    
    fg_locks = folium.FeatureGroup(name="Locks and Dam",overlay=True,show=False)
    fg_other = folium.FeatureGroup(name="Other Sites",overlay=True,show=False)

    for i, row in sites.iterrows():
        coord = [row.geometry.y,row.geometry.x]
        label = row.Type+' @ '+row.Name
        marker_type = 'pin'
        radius = 3
        color = 'red'
        # Available colors: red, darkred, lightred, pink, green, darkgreen, lightgreen, blue, darkblue, lightblue, cadetblue, purple, darkpurple, orange, beige, black, gray, lightgray
        icon = 'dashboard'
        html = label
        width = 500
        height = 120
        max_width = 1000
        
        if (row.Type == 'Locks and Dam'):
            marker_type = 'circle'
            radius = 5
            color = 'red'
            fill = True
            fill_color = color
            html += '<br><a href=\"https://www.lrh.usace.army.mil/Missions/Civil-Works/Locks-and-Dams/{0:s}'.format(row['USACE Code'])+'\",target=\"_blank\">USACE Data for Locks and Dams at {0:s}'.format(row['Name'])+'</a>'
            
            if (row.Name == 'Greenup'):
                html += '<br><a href=\"https://v2.wqdatalive.com/project/view/1010/?d=1168\",target=\"_blank\">WQData Live data at {0:s} Locks and Dams'.format(row['Name'])+'</a>'
                html += '<br>{0:s}'.format(str(row.Description))
                html += '<br>Lat: {0:.2f}'.format(row.geometry.y)+', Lon: {0:.2f}'.format(row.geometry.x)
                html += '<br><iframe src="https://v2.wqdatalive.com/project/applet/html/1010?d=1168&refresh=true" allowtransparency=true style="width: 300px; height: 420px; overflow-y: hidden; overflow-x: hidden;"></iframe>'
                iframe = folium.IFrame(html,width=width,height=height)
                popup = folium.Popup(iframe,max_width=max_width)
                
                if (marker_type == 'pin'):
                    fg_locks.add_child(folium.Marker(location=coord,icon=folium.Icon(color=color,icon=icon),popup=popup,tooltip=label)).add_to(river_map)
                elif (marker_type == 'circle'):
                    fg_locks.add_child(folium.CircleMarker(location=coord,radius=radius,color=color,popup=popup,tooltip=label)).add_to(river_map)
                    
            elif (row.Name == 'RC Byrd'):
                html += '<br><a href=\"https://v2.wqdatalive.com/project/view/1010/?d=1169\",target=\"_blank\">WQData Live data at {0:s} Locks and Dams'.format(row['Name'])+'</a>'
                html += '<br>{0:s}'.format(str(row.Description))
                html += '<br>Lat: {0:.2f}'.format(row.geometry.y)+', Lon: {0:.2f}'.format(row.geometry.x)
                html += '<br><iframe src="https://v2.wqdatalive.com/project/applet/html/1010?d=1169&refresh=true" allowtransparency=true style="width: 300px; height: 420px; overflow-y: hidden; overflow-x: hidden;"></iframe>'
                iframe = folium.IFrame(html,width=width,height=height)
                popup = folium.Popup(iframe,max_width=max_width)
                
                if (marker_type == 'pin'):
                    fg_locks.add_child(folium.Marker(location=coord,icon=folium.Icon(color=color,icon=icon),popup=popup,tooltip=label)).add_to(river_map)
                elif (marker_type == 'circle'):
                    fg_locks.add_child(folium.CircleMarker(location=coord,radius=radius,color=color,popup=popup,tooltip=label)).add_to(river_map)
                    
        else:
            marker_type = 'circle'
            radius = 3
            color = 'blue'
            fill = True
            fill_color = color
            html += '<br>{0:s}'.format(str(row.Description))
            html += '<br>Lat: {0:.2f}'.format(row.geometry.y)+', Lon: {0:.2f}'.format(row.geometry.x)
            iframe = folium.IFrame(html,width=width,height=height)
            popup = folium.Popup(iframe,max_width=max_width)
            
            if (marker_type == 'pin'):
                fg_other.add_child(folium.Marker(location=coord,icon=folium.Icon(color=color,icon=icon),popup=popup,tooltip=label)).add_to(river_map)
            elif (marker_type == 'circle'):
                fg_other.add_child(folium.CircleMarker(location=coord,radius=radius,color=color,popup=popup,tooltip=label)).add_to(river_map)

### Add built-in basemaps

In [148]:
folium.TileLayer('OpenStreetMap').add_to(river_map)
folium.TileLayer('StamenTerrain').add_to(river_map)

### Add other basemap options using Map Servers at  ArcGIS Online

In [149]:
mapserver_query = '/MapServer/tile/{z}/{y}/{x}'
mapserver_dict = dict(NatGeo_World_Map=ARCGIS_WS+'/NatGeo_World_Map/MapServer',
            World_Imagery=ARCGIS_WS+'/World_Imagery/MapServer',
            World_Shaded_Relief=ARCGIS_WS+'/World_Shaded_Relief/MapServer',
            World_Street_Map=ARCGIS_WS+'/World_Street_Map/MapServer',
            World_Topo_Map=ARCGIS_WS+'/World_Topo_Map/MapServer',
#            World_Physical_Map=ARCGIS_WS+'/World_Physical_Map/MapServer',
            World_Terrain_Base=ARCGIS_WS+'/World_Terrain_Base/MapServer')

for tile_name, tile_url in mapserver_dict.items():
    tile_url += mapserver_query
    folium.TileLayer(tile_url,name=tile_name,attr=tile_name).add_to(river_map)

### Add the Layer Control widget

In [150]:
folium.LayerControl().add_to(river_map)

### Save the map as HTML
 * Suppress before publishing as read-only Tool

<div id='map' />

In [151]:
river_map.save(OUT_DIR+'/river_map.html')

## Display Map

In [152]:
from IPython.display import IFrame
IFrame(OUT_DIR+"/river_map.html", width=1500, height=750)